# Ensemble1
앙상블 기법 voting, bagging, random forest에 대한 실습을 진행합니다.  
폐암 진단 데이터를 이용한 분류기 만들기 앙상블입니다.  

In [1]:
# ready
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
# sklearn 

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB     
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## 1. Voting : hard & soft
sklearn 내장 데이터인 breast cancer dataset을 사용했습니다.  
2주차와 3주차에 배웠던 머신러닝 분류모델들을 모두 사용해보겠습니다.

In [2]:
# load dataset
cancer = load_breast_cancer()
data = pd.DataFrame(cancer.data, columns=cancer.feature_names)
data.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [3]:
X,y = cancer.data, cancer.target  
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, test_size=0.2, random_state=1004)

In [4]:
y_train[:20]  #cancer 여부

array([0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0])

* 개별 모델의 성능

In [5]:
#Logistic Regression
lr = LogisticRegression()
lr.fit(X_train,y_train)
pred = lr.predict(X_test)
print('test acurracy : {0:4f}'.format(accuracy_score(y_test, pred)))

test acurracy : 0.912281


In [6]:
#Decision Tree
dt = DecisionTreeClassifier(random_state=318)
dt.fit(X_train,y_train)
pred = dt.predict(X_test)
print('test acurracy : {0:4f}'.format(accuracy_score(y_test, pred)))

test acurracy : 0.885965


In [7]:
knn = KNeighborsClassifier(n_neighbors=9)
knn.fit(X_train, y_train)
pred = knn.predict(X_test)
print('test acurracy : {0:4f}'.format(accuracy_score(y_test, pred)))

test acurracy : 0.894737


In [8]:
#SVM Classifier
svc = SVC(probability=True)  #SVM for Regression : SVR
svc.fit(X_train,y_train)
pred = svc.predict(X_test)
print('test acurracy : {0:4f}'.format(accuracy_score(y_test, pred)))

test acurracy : 0.868421


* Voting 모델의 성능

In [9]:
#Voting
from sklearn.ensemble import VotingClassifier

# base model 설정
lr = LogisticRegression()
dt = DecisionTreeClassifier(random_state=318)
knn = KNeighborsClassifier(n_neighbors=9)

# 앙상블 모델 - hard, soft 설정
vo_hard = VotingClassifier(estimators=[("LR",lr),("dt",dt),("KNN",knn)],voting="hard") 
vo_soft = VotingClassifier(estimators=[("LR",lr),("dt",dt),("KNN",knn)],voting="soft")

# VotingClassifier 학습/예측/평가
vo_hard.fit(X_train,y_train)
pred_h = vo_hard.predict(X_test)
print('Hard Voting 분류기 정확도: {0:4f}'.format(accuracy_score(y_test, pred_h)))

vo_soft.fit(X_train,y_train)
pred_s = vo_soft.predict(X_test)
print('Soft Voting 분류기 정확도: {0:4f}'.format(accuracy_score(y_test, pred_s)))

Hard Voting 분류기 정확도: 0.921053
Soft Voting 분류기 정확도: 0.929825


미세하게 soft voting으로 만든 앙상블 모델이 가장 좋은 성능을 가집니다.

Naive Bayes, SVM 모델로 Voting
* 개별 모델 성능

In [10]:
nb = GaussianNB()
nb.fit(X_train, y_train)
pred = nb.predict(X_test)
print('test acurracy : {0:4f}'.format(accuracy_score(y_test, pred)))

test acurracy : 0.903509


In [11]:
svc = SVC(probability=True)
svc.fit(X_train, y_train)
pred = svc.predict(X_test)
print('test acurracy : {0:4f}'.format(accuracy_score(y_test, pred)))

test acurracy : 0.868421


* Voting모델 성능

In [12]:
# svc 추가
# base model 설정
nb = GaussianNB()
svc = SVC(probability=True)

# 앙상블 모델 - hard, soft 설정
vo_hard = VotingClassifier(estimators=[("nb",nb),("svc",svc)],voting="hard")
vo_soft = VotingClassifier(estimators=[("nb",nb),("svc",svc)],voting="soft")

# VotingClassifier 학습/예측/평가
vo_hard.fit(X_train,y_train)
pred_h = vo_hard.predict(X_test)
print('Hard Voting 분류기 정확도: {0:4f}'.format(accuracy_score(y_test, pred_h)))

vo_soft.fit(X_train,y_train)
pred_s = vo_soft.predict(X_test)
print('Soft Voting 분류기 정확도: {0:4f}'.format(accuracy_score(y_test, pred_s)))

Hard Voting 분류기 정확도: 0.912281
Soft Voting 분류기 정확도: 0.885965


나이브베이즈와 svm 으로 만든 앙상블 모델은 hard voting이 성능이 제일 좋았습니다. 무조건 soft가 더 성능이 좋은 것은 아님을 알 수 있습니다.

# 2. Bagging

Scikit-Learn 의 ensemble 서브패키지는 배깅 모형 결합을 위한 BaggingClassifier() 클래스를 제공합니다. 파라미터의 의미는 다음과 같습니다.  

- base_estimator: 기본 모델 지정. 디폴트 Decision Tree  
- n_estimators: 모델 갯수. 디폴트 10  
- bootstrap: 데이터의 중복 사용 여부. 디폴트 True  
- max_samples: 데이터 샘플 중 선택할 샘플의 수. 디폴트 1.0  
- max_features: 다차원 독립 변수 중 선택할 변수의 수 1.0  
- bootstrap_features: 특징 차원의 중복 사용 여부. 디폴트 False  


In [13]:
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

* Decision tree를 이용하여 Bagging 수행

In [14]:
# model
clf = DecisionTreeClassifier(random_state=318)
eclf = BaggingClassifier(clf,oob_score=True)

In [15]:
# cv
cross_val_score(eclf,X,y,cv=5).mean()

0.9543704393727681

Grid search로 최적의 파라미터를 찾습니다.
* estimator: 학습이 안된 빈 모델 (이것저것 시도해봐야 하니까)
* param_grid: "Dictionary with parameters", 시도해보고 싶은 파라미터 목록을 딕셔너리 형태로
* cv: cross validation split 수, Kfold로도 대체 가능, 디폴트 5
* n_jobs: Number of jobs to run in parallel, -1 means using all processors
* refit : 최적의 파라미터로 다시 모델을 피팅할것인지의 여부. 디폴트 True

결과 관련 attributes
* best_params_ : 최적의 파라미터
* best_score_ : 최적의 파라미터에서의 성능 (최적의 성능)
* best_estimator_ : 최적의 파라미터로 refit한 모델

In [16]:
# Grid search
bagging = BaggingClassifier(clf,oob_score=True) # 빈 모델
params ={
    "n_estimators" : [10,20,30,40,50,55],
    "max_samples" : [0.5,0.6,0.7,0.8,0.9,1]}

grid = GridSearchCV(estimator=bagging, param_grid=params, cv=5, n_jobs=-1)
grid = grid.fit(X, y)

In [ ]:
# best score & params
print("best score: ", grid.best_score_)  #최적의 스코어
print("best parameters: ", grid.best_params_) #최적의 파라미터

# oob score
print("oob score: ", grid.best_estimator_.oob_score_) #최적의 파라미터로 설정된 모델에서의 oob score

best score:  0.9718987734823784
best parameters:  {'max_features': 4, 'n_estimators': 30}
oob score:  0.9595782073813708


단순 voting 과 비교했을 때(voting은 0.88~0.91), 튜닝을 진행한 bagging 모델이 성능이 월등히 좋아보입니다.

# 3. Random Forest

In [ ]:
# model
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, max_features=2, n_jobs=-1, oob_score=True)

In [ ]:
# cv
cross_val_score(rf, X, y, cv=5).mean()

0.9648657040832168

In [ ]:
# Grid search
rf2 = RandomForestClassifier(n_estimators=100, max_features=2, n_jobs=-1, oob_score=True)
params ={
    "n_estimators" : [10, 20, 30, 50, 100],
    "max_features" : [1,2,3,4,5,6,7, 10, 15, 20, 25, len(X[0])]
    }

grid = GridSearchCV(estimator=rf, param_grid=params, cv=5, n_jobs=-1)
grid = grid.fit(X, y)

In [ ]:
# best score & params
print("best score: ", grid.best_score_)  
print("best parameters: ", grid.best_params_)

# oob score
print("oob score: ",grid.best_estimator_.oob_score_)

best score:  0.9718987734823784
best parameters:  {'max_features': 4, 'n_estimators': 30}
oob score:  0.9595782073813708


Feature Importances
* impurity-based feature importances.(Gini importance) 높을수록 더 중요한 feature
* 해당 feature가 얼마나 impurity를 개선하는지

In [ ]:
grid.best_estimator_.feature_importances_  #데이터의 feature(column) 수가 30개. 각 feature의 중요도를 나타냄

array([0.09024372, 0.02257772, 0.06606393, 0.01196214, 0.00804551,
       0.00749603, 0.03796903, 0.10023773, 0.00790435, 0.00311315,
       0.01804004, 0.00383068, 0.01022444, 0.04610047, 0.00450703,
       0.00368308, 0.00363821, 0.00268246, 0.00251732, 0.00237052,
       0.093805  , 0.01881718, 0.16020502, 0.0872881 , 0.01783188,
       0.00460009, 0.03080236, 0.11489141, 0.00913497, 0.00941643])

In [ ]:
np.argsort(grid.best_estimator_.feature_importances_)[::-1] 

array([22, 27,  7, 20,  0, 23,  2, 13,  6, 26,  1, 21, 10, 24,  3, 12, 29,
       28,  4,  8,  5, 25, 14, 11, 15, 16,  9, 17, 18, 19])

중요도가 높은 것부터 인덱스 정렬. 인덱스 22가 0.16020502로 중요도가 가장 높고, 인덱스 27이 0.11489141로 두번째로 높음